Collects redirects from the xml dump directly, further scraping from the xml dump is not needed.
As The scrape_wiki.py and scrape_nowiki_1.ipynb does the job of filling out the data fields of an adequate amount of wikipedia pages.

In [1]:
import tarfile
import glob
import os

import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os

import pandas as pd


In [2]:
def extract_from_xml(xml_tree):
    article = ""
    title = ""
    url = ""
    date = ""
    classes = ""

    div_elms = xml_tree.findall("*//div")
    for elm in div_elms:
        div_type = elm.attrib.get("type")
        if div_type == "ingress":
            ingress = elm.text.strip() if elm.text else None
        elif div_type == "text":
            sentences = elm.findall("p")
            article += " ".join(sentence.text.strip() for sentence in sentences)
        elif div_type == "title" and elm.attrib.get("level") == "1":
            title = elm.text.strip()

    if not article or not ingress:
        return None

    attr_elms = xml_tree.findall("*//attribute")
    for elm in attr_elms:
        attr_name = elm.attrib.get("name")
        if attr_name == "url":
            url = elm.attrib.get("value")
        elif attr_name == "date":
            date = elm.attrib.get("value")
        elif attr_name == "class1":
            classes = elm.attrib.get("value")


    return {"title": title, "url": url, "published": date, "classes": classes, "article": article, "ingress": ingress}


In [3]:
PATH_WIKI_XML = '.'
FILENAME_WIKI = 'nowiki-20230120-pages-articles.xml'
FILENAME_ARTICLES = 'articles1.csv'
FILENAME_REDIRECT = 'articles_redirect1.csv'
FILENAME_TEMPLATE = 'articles_template1.csv'
ENCODING = "utf-8"

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

pathWikiXML = os.path.join(PATH_WIKI_XML, FILENAME_WIKI)
pathArticles = os.path.join(PATH_WIKI_XML, FILENAME_ARTICLES)
pathArticlesRedirect = os.path.join(PATH_WIKI_XML, FILENAME_REDIRECT)
pathTemplateRedirect = os.path.join(PATH_WIKI_XML, FILENAME_TEMPLATE)

totalCount = 0
articleCount = 0
redirectCount = 0
templateCount = 0
title = None
start_time = time.time()

In [4]:
nowiki_dataset = pd.DataFrame(columns=["title", "url", "published", "classes", "article", "ingress"])

with codecs.open(pathArticles, "w", ENCODING) as articlesFH, \
        codecs.open(pathArticlesRedirect, "w", ENCODING) as redirectFH, \
        codecs.open(pathTemplateRedirect, "w", ENCODING) as templateFH:
    articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)
    redirectWriter = csv.writer(redirectFH, quoting=csv.QUOTE_MINIMAL)
    templateWriter = csv.writer(templateFH, quoting=csv.QUOTE_MINIMAL)

    articlesWriter.writerow(['id', 'title', 'redirect'])
    redirectWriter.writerow(['id', 'title', 'redirect'])
    templateWriter.writerow(['id', 'title'])
    for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
        tname = strip_tag_name(elem.tag)

        if event == 'start':
            if tname == 'page':
                title = ''
                id = -1
                redirect = ''
                inrevision = False
                ns = 0
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
        else:
            if tname == 'title':
                title = elem.text
            elif tname == 'id' and not inrevision:
                id = int(elem.text)
            elif tname == 'redirect':
                redirect = elem.attrib['title']
            elif tname == 'ns':
                ns = int(elem.text)
            elif tname == 'page':
                totalCount += 1

                if ns == 10:
                    templateCount += 1
                    templateWriter.writerow([id, title])
                elif len(redirect) > 0:
                    articleCount += 1
                    articlesWriter.writerow([id, title, redirect])
                else:
                    redirectCount += 1
                    redirectWriter.writerow([id, title, redirect])
                    
        if totalCount > 1 and (totalCount % 100000) == 0:
            print("{:,}".format(totalCount))
        #print(tname)

elem.clear()

elapsed_time = time.time() - start_time

print("Total pages: {:,}".format(totalCount))
print("Template pages: {:,}".format(templateCount))
print("Article pages: {:,}".format(articleCount))
print("Redirect pages: {:,}".format(redirectCount))
print("Elapsed time: {}".format(hms_string(elapsed_time)))

FileNotFoundError: [Errno 2] No such file or directory: '.\\nowiki-20230120-pages-articles.xml'

In [5]:
with open(pathWikiXML, 'r', encoding=ENCODING) as f:
    row = f.readline()
    #print(row)
    while row:
        row = f.readline()
        #print(row)
    

FileNotFoundError: [Errno 2] No such file or directory: '.\\nowiki-20230120-pages-articles.xml'